In [1]:
alphabeth = 'abcdefghijklmnopqrstuvwxyz, '
# the alphabet can be changed to more detailed one(for example with capital letters, numbers, special characters etc.)

One of the main steps in Ceasar cypher is to move each letter by a fixed number of places in the alphabeth using the key.

This `moveLetter` function is used to move one letter by a fixed number of places(key) in the alphabeth.

Key can be negative, meaning that the letter will be moved to the left in the alphabeth instead of to the right.


In [2]:
def moveLetter(letter,key):
    # find the index of the given letter in the alphabeth defined above
    letterIndex = alphabeth.find(letter)
    # find the index of the new letter by adding the key to the index. The new index can go out of range of the alphabeth(e.g. 27, -10)
    # so we use the modulo operator to get the proper index.
    newLetterIndex = (letterIndex+key) % len(alphabeth) 
    return alphabeth[newLetterIndex]

The `encryptMessage` function takes message and the key, then using `for` loop it iterates over each letter in the message and calls `moveLetter` function to move the letter by the key.

The results is stored in `encryptedMessage` variable.


In [3]:
def encryptMessage(message, key):
    encryptedMessage = ""
    for letter in message:
        encryptedMessage+=moveLetter(letter,key)
    return encryptedMessage

And out Ceasar cypher is ready to use!

Let's get the ecrypted message and a key as an input from the user and print the encrypted message.

In [4]:
usePrefefinedData = input("Do you want to use predefined data? (y/n)\n")
if usePrefefinedData == "y":
    message = "test message, the quick brown fox jumps over the lazy dog"
    key = 13
else:
    message = input('What is the message you want to encrypt?\n')
    key = input("Enter your key to encrypt with\n")

encryptedMessage = encryptMessage(message, int(key))
print(f"Message before the encryption: {message}\nThe encryptedMessage: {encryptedMessage}")

Message before the encryption: test
The encryptedMessage: dqcd


The decryption is done by using the same `moveLetter` function, but using a negative key, moving letters to the left by the key value and retrieving the original message.

In [5]:
def decryptMessage(encryptedMessage, key):
    decryptedMessage = ""
    for letter in encryptedMessage:
        decryptedMessage+=moveLetter(letter,-key)
    return decryptedMessage

Now let's write bruteforce decryption function. We take the encrypted message as the input and test all possible keys from 1 to the length of the alphabeth-1($keys = [1.. (|alphabeth|-1)$). 

We use the keys for `decryptMessage` function and print the results.

One of printed results will be the original message.

In [6]:
def bruteForceCeasarCypher(message):
    for key in range(1,len(alphabeth)):
        print(decryptMessage(message,key))

In [7]:
import random

useOldMessage = input("Do you want to use the old message? (y/n)\n")
if useOldMessage == "n":
    message = input('What is the message you want to encrypt?\n')
    key = random(1,len(alphabeth))
    encryptedMessage = encryptMessage(message, key)
    
print(f"Message before the encryption: {message}\nThe encryptedMessage: {encryptedMessage}")
print("Let's brute force it\nAll possible messages are:\n")
bruteForceCeasarCypher(encryptedMessage)

TypeError: 'module' object is not callable

#### Please use this file to write and execute your solution.



Write a python code in cocalc that breaks the following using brute force attack:

Assume you got a ciphertext (in hexa) that  is  b'1525053514291239',  as a result of XOR encryption.
Additionally, assume that the unknown plaintext is an 8-letter word and among the most common words in English
You know that a very weak key is used that consists of a repeated character followed by @ sign. For example, if the word is an 8 letter word, then the key will be: ?@?@?@?@
Your task is to brute force this ciphertext based on the above information.
Output any successful decrypted plaintext, if you have found it among the most common words in English.

In [9]:
# Auxiliary func:
from binascii import hexlify, unhexlify

In [10]:
def xor(string1, string2):
    return "".join([chr(ord(c1) ^ ord(c2)) for (c1,c2) in zip(string1,string2)])

In [11]:
cipher_hex = b'1525053514291239'

Let's create generic generate key function that takes a key length and a character to repeat. This function will return a 'char' + '?' repeated until the key length is reached. For example, if the key length is 8 and the character is 'a', then the key will be 'a?a?a?a?'. 

In [12]:
def generateKey(length,code):
    letter = [x for x in (chr(code) * length)] # generate key of length 'length' with character 'code' and split it into a list
    return "@".join(letter)[:length] # join the list with '@' and return the first 'length' characters

We create function to load the most common words in English from the file 'common_words.txt' and return the list of them.

In [13]:
def loadDictionary():
    with open('common_words.txt') as f:
        return f.read().splitlines()

The 'brute_crack' function takes cipher_text. It will iterate through the 256 ASCII letters generateing weak key for them("?@?@..."). 

Then it will use XOR with the key and unhexlified cipher_hex and check if the result is in the list of common words.

 If it is, then it will return the key and the message.

In [14]:
def brute_crack(cipher_hex):
    dictionary = loadDictionary()
    c = unhexlify(cipher_hex).decode()
    for i in range(256):
        k = generateKey(len(c),i)
        m = xor(c, k)
        if m in dictionary:
            return (k, m)


In [15]:
(key, message) = brute_crack(cipher_hex)
print(f"key: {key}")
print(f"message: {message}")

key: f@f@f@f@
message: security


Write a python code in cocalc that crack the used random number generator, based on the following scenario:

Assume you got a ciphertext (in hexa) that  is  b'62716b6b777862677362667e7b',  as a result of XOR encryption.
Additionally, assume that you know the original encrypted plaintext was 'cybersecurity'
You know that the key were random numbers generated by LCG.
By knowing the above information, crack the LCG parameters.
 


# Imports

In [16]:
from functools import reduce
from binascii import unhexlify

# Setup

we use `xor`, `egcd`, `modinv` `gcd` from the previous lectures.

In [17]:
def xor(string1, string2):
    return "".join([chr(ord(c1) ^ ord(c2)) for (c1,c2) in zip(string1,string2)])

In [18]:
def egcd(a,b):
    if a ==0 :
        return (b,0,1)
    else:
        g,y,x = egcd(b%a,a)
        return (g,x-(b//a)*y,y)
    
def modinv(b,n):
    g,x,_ = egcd(b,n)
    if g == 1:
        return x%n

In [19]:
def gcd(x,y):
    while y:
        x,y = y, x%y
    return x

# LCG cracking algorithm
We use the algorithm from the lecture slides. The algorithm is as follows:

1. Compute the modulus of the LCG using the key values.
2. Compute the multiplier of the LCG using the key values and modulus.
3. Compute the increment of the LCG using the key values, modulus, and multiplier.

In [20]:
def crack_unknown_increment(states, modulus, multiplied):
    increment = (states[1]-states[0]*multiplied) % modulus
    return increment

In [21]:
def crack_unknown_multiplier(states, modulus):
    multiplier = (states[2]-states[1]) * modinv(states[1]-states[0],modulus) % modulus
    increment = crack_unknown_increment(states, modulus, multiplier)
    return increment, multiplier

In [22]:
def crack_unknown_modulus(states):
    diffs = [s1 - s0 for s0, s1 in zip(states, states[1:])]
    zeroes = [t2*t0 - t1*t1 for t0, t1, t2 in zip(diffs, diffs[1:], diffs[2:])]
    modulus = abs(reduce(gcd, zeroes))
    increment, multiplier = crack_unknown_multiplier(states, modulus)
    return modulus, increment, multiplier

# Solution
We get the cyphertext and plaintext from the problem statement. We unhexify the cyphertext. We then get the key by XORing the cyphertext and plaintext. Assuming that the key is generated by the LCG, we can use the key to crack the LCG parameters.

In [23]:
def getLCGValues(cypher_hex,message):
    unhexifiedCypherText = unhexlify(cypher_hex).decode()
    print("unhexified cypher text: ",unhexifiedCypherText)
    values = xor(unhexifiedCypherText, message)
    keys = [ord(c) for c in values]
    print(f"Keys used for encrypting {message}:",keys)
    return crack_unknown_modulus(keys)

### Getting the LCG values


In [24]:
cypher_hex = b'62716b6b777862677362667e7b'
message = "cybersecurity"
(modulus,increment, multiplier) = getLCGValues(cypher_hex,message)

print("modulus: ", modulus, "\nincrement: ", increment, "\nmultiplier: ", multiplier)

unhexified cypher text:  bqkkwxbgsbf~{
Keys used for encrypting cybersecurity: [1, 8, 9, 14, 5, 11, 7, 4, 6, 16, 15, 10, 2]
modulus:  17 
increment:  3 
multiplier:  5


## Testing
After we got our LCG modulus, increment, multiplier values, we can check if those values actually generate "random" sequence used in the encryption of the message

We use a bit modified LCG algorithm where the first state is the seed itself and we pass the LCG parameters as an arguments of a function.

In [25]:
def prng_lcg(seed, repeat, multiplier, increment, modulus):
    state = seed
    numbers = [seed]
    for _ in range(repeat-1):
        state = (state * multiplier+increment) % modulus
        numbers.append(state)
    return numbers

In [26]:
generatedKeyList = prng_lcg(1,len(message),multiplier,increment,modulus)
print (generatedKeyList)

[1, 8, 9, 14, 5, 11, 7, 4, 6, 16, 15, 10, 2]


Let's compare our generated Key list and key list we got after using xor for cyphertext and message.

In [27]:
unhexifiedCypherText = unhexlify(cypher_hex).decode()
values = xor(unhexifiedCypherText, message)
originalKeyList = [ord(c) for c in values]

In [28]:
def checkIfEqualKeyLists(originalKeyList, newKeyList):
    areEqual = True
    for (ind,_) in enumerate(originalKeyList):
        if originalKeyList[ind] != newKeyList[ind]:    
            areEqual = False
    if areEqual:
        print("The lists are equal")
    else:
        print("The lists are not equal")

In [29]:
checkIfEqualKeyLists(originalKeyList,generatedKeyList)

The lists are equal


Write a python code in cocalc that implement the following scenario:

1) Read a stream of data from (data_stream.txt) file, one character at a time.

2) Generate the required number of bits using LFSR (generates 8-bit at a time for each character)

3) Perform the xor encryption.

4) In a separate function perform the xor decryption. Note: during decryption the sequence of key can be generated using the seed and mask value

 

5) OPTIONAL: this is an optional step. Try to crack the LFSR, by knowing the original data stream and the ciphertext.



LRSR generation function(from the lecture slides):

In [30]:
def LRSR_generate(seed, mask, n):
    seed_int = int(seed, 2)
    mask_int = int(mask, 2)
    nbits = len(seed)

    state = seed_int
    while n > 0:
        output = 1 & state
        _mask,_state,new_bit = mask_int, state, 0
        while _mask:
            new_bit ^= (1&_mask) * (1&_state)
            _mask >>= 1
            _state >>= 1
        state = state >> 1 | (new_bit << (nbits-1))

        yield output, state
        n-=1

Let's split the list of integers into 8 element chunks. The `list_split` function will do that for us.

In [31]:
def list_split(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [32]:
def convertListOfBinaryToChar(list):
    return chr(int("".join(map(str, list)),2))

To convert a list of integers into a string, we can `getKeyAsString` function. First we split the list into 8 element chunks, then we convert each chunk into a char and join the chars into a string. 

In [33]:
def getKeyAsString (key):
    keyLists = list(list_split ([ output for output, _ in key],8))
    return ''.join(convertListOfBinaryToChar(binaryList) for binaryList in keyLists)

Xor function(from earlier laboratories):

In [34]:
def xor(string1, string2):
    return "".join([chr(ord(c1) ^ ord(c2)) for (c1,c2) in zip(string1,string2)])

Read one character at a time from the file function. Reads using `read(1)` function(returns one character at a time)

In [35]:
def readFromFileOneCharAtATime(filename):
    with open(filename, "r") as f:
        while True:
            c = f.read(1)
            if not c:
                break
            yield c

Data stream file:

In [36]:
file = "data_stream.txt"

A small streamEncryption function, which takes filename, seed and mask as an input and returns xor encrypted data using LFSR function

In [37]:
def streamEncrypt(file,seed,mask):
    text = ''.join(str(output) for output in readFromFileOneCharAtATime(file))
    key = LRSR_generate(seed, mask, len(text) * 8)
    key_str = getKeyAsString(key)
    print (key_str)
    # print (key_str)
    print(key_str)
    encrypted_text = xor(text, key_str)
    return encrypted_text

In [38]:
# # seed = input('Enter seed: ')
# mask = input('Enter mask: ')
seed ='1001011001'
mask ='0101101101'

encrypted_text = streamEncrypt(file,seed,mask)
print(encrypted_text)

dC3Èr+ìvæPØ9¹`Ã½GÐ:Éoæ÷4Ègå$WØíÌ¡°2r5rÁz
dC3Èr+ìvæPØ9¹`Ã½GÐ:Éoæ÷4Ègå$WØíÌ¡°2r5rÁz
î*@èáV$ª|Xw¥#uk±%N¦Oî×õF©èöV%½ìÒÕQG­þT


A `streamDecprypt` function which takes encrypted text, seed and mask as an input and returns the decrypted text by using the LFSR function to generate the key and xor the encrypted text and key to retrieve the original text.

In [39]:
def streamDecrypt(encrypted_text, seed, mask):
    key = LRSR_generate(seed, mask, len(encrypted_text)*8)
    key_str = getKeyAsString(key)
    originalText = xor(encrypted_text, key_str)
    return originalText

In [40]:
print(streamDecrypt(encrypted_text, seed, mask))

this is a stream of data to be transferred securely.


Let's crack LRSR

In [41]:
def retrieveKey(encrypted_text, original_text):
    key = xor(encrypted_text, original_text)
    return key


original_text = ''.join(str(output)
                        for output in readFromFileOneCharAtATime(file))
retrievedKey = retrieveKey(encrypted_text, original_text)


Get binary string from the given key in char format

In [42]:
def getBinaryValues(key): ##('0100')
    keys=list(key)
    ones = [format(ord(k),'b').zfill(8) for k in keys]
    return ''.join(i for i in ones)

Let's try to get mask value from the key value. We can do this using berlekam massey algorithm:

In [43]:
import berlekampmassey

def GetMaskFromKey(key):
    binaryValues = getBinaryValues(key)
    ret = list(berlekampmassey.bm(binaryValues))[-1]
    return ''.join([str(x) for x in ret[::-1]])

Since the seed is inside the key, but don't know its length, we can iterate from 1 till the length of the key to find the seed value. Adter choosing `i` length seed from the key, we generate key length of values using LFSR and check if the generated key is equal to the given key. If it is equal, we found the seed value.

In [44]:
def GetSeedFromKey(key, mask):
    keyInBinary = getBinaryValues(key)
    for i in range(1, len(keyInBinary)):
        tmp_seed = keyInBinary[0:i][::-1]
        print(tmp_seed)
        tmp_key = LRSR_generate(tmp_seed, mask, len(keyInBinary)*16)
        tmp_key_str = ''.join(str(output) for output, _ in tmp_key)

        if (keyInBinary == tmp_key_str):
            return tmp_seed

Full seed, mask retrieval from the key:

In [45]:
solvedMask = GetMaskFromKey(retrievedKey)
print("mask comparison")
print("original mask")
print(mask)
print("retrieved mask from the key")
print(solvedMask)
print('\nseed comparison')
print("original seed")
print(seed)
print("retrieved seed from the key with solved mask")
print(GetSeedFromKey(retrievedKey, solvedMask))

mask comparison
original mask
0101101101
retrieved mask from the key
0101101101

seed comparison
original seed
1001011001
retrieved seed from the key with solved mask
1
01
001
1001
11001
011001
1011001
01011001
001011001
1001011001
11001011001
011001011001
0011001011001
10011001011001
010011001011001
0010011001011001
00010011001011001
100010011001011001
0100010011001011001
00100010011001011001
000100010011001011001
0000100010011001011001
10000100010011001011001
110000100010011001011001
0110000100010011001011001
00110000100010011001011001
100110000100010011001011001
1100110000100010011001011001
01100110000100010011001011001
001100110000100010011001011001
1001100110000100010011001011001
11001100110000100010011001011001
111001100110000100010011001011001
1111001100110000100010011001011001
01111001100110000100010011001011001
001111001100110000100010011001011001
1001111001100110000100010011001011001
01001111001100110000100010011001011001
001001111001100110000100010011001011001
...
None


Create a file called message.txt with some content of your choice. Then, write a python code in cocalc that implement the following scenario:

1) Create a sender function the does the following:

- Read the content of the file message.txt.
- Hash the content using SHA1 hash function.
- Encrypt the content using AES-CBC encryption algorithm
- The output will be both the hash value of the message, and the ciphertext.

2) Create a receiver function the does the following:

- The input is the original hash value, and the ciphertext.
- Decrypt the received ciphertext using AES-CBC.
- Calculate the hash value of the decrypted ciphertext
- Compare the original hash value with the hash value of the decrypted ciphertext.

Needed imports for the block cypher exercise

In [46]:
from hashlib import sha1
from Crypto.Cipher import AES
from Cryptodome.Random import get_random_bytes
import secrets

Reading the file content function

In [47]:
def readFileConent(fileName):
    with open(fileName, 'r',encoding='utf-8') as f:
        return f.read()

SHA 1 hash function

In [48]:
def hashFileContentWithSha1(fileContent):
    return sha1(fileContent.encode()).hexdigest()

padding from the cryptography slides

In [49]:
BLOCK_SIZE = 16
pad = lambda s : s +(BLOCK_SIZE-len(s)%BLOCK_SIZE)*chr(BLOCK_SIZE-len(s)%BLOCK_SIZE)
unpad = lambda s : s[:-ord(s[len(s)-1:])]

Encrypting the file content with AES-CBC

In [50]:
def encryptTheContentWithAESCBC(fileContent, key):
    iv = secrets.token_bytes(BLOCK_SIZE)
    enc = AES.new(key, AES.MODE_CBC, iv)
    data = pad(fileContent).encode("utf-8")
    cipherText = enc.encrypt(data)
    cipherTextHex = iv.hex() + cipherText.hex()
    return cipherTextHex

encrypting file content given key and filename and returning hash value and ciphertext

In [51]:
def encryptContent (fileName, key):
    content = readFileConent(fileName)
    print(content)
    hash = hashFileContentWithSha1(content)
    return (hash,encryptTheContentWithAESCBC(content, key))

In [52]:
key = get_random_bytes(BLOCK_SIZE)
hash, cipherTextHex = encryptContent('message.txt', key)
print("hash: ",hash)
print("cipher text: ", cipherTextHex)

hello world from Budapest

hash:  5ba2eef200f737dbfbd6f564315503d518b3686d
cipher text:  bddecab27783515bbdf7a38cf74ccaff2403a2f6f1dcb99ee11fa0e8784c0616ddcc02fbb50d63d3c4078128d44e1083


Decrypting the file content with AES-CBC, and comparing the hash value of decrypted text with the original hash value of the file content.

In [53]:
def decryptAES(hash, cipherText):
    iv = bytes.fromhex(cipherText[:32])
    cipherText = bytes.fromhex(cipherText[32:])
    dec = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(dec.decrypt(cipherText)).decode('utf-8')
    hashValue = hashFileContentWithSha1(plaintext)
    print('decrypted value: ', plaintext)
    if hashValue == hash:
        print ("Hash is equal")
    else:
        print ("Hash is not equal")


In [54]:
decryptAES(hash, cipherTextHex)

decrypted value:  hello world from Budapest

Hash is equal


Create a file called message.txt with some content of your choice. Then, write a python code in cocalc that implement the following scenario:

Prior to the following function, you need to prepare the parameters to be used for RSA encryption by the sender. These include the public and private exponents, and the integer n.

 

1) Create a sender function the does the following:

Read the content of the file message.txt.
Hash the content using SHA1 hash function.
Encrypt the hash of the message.txt using RSA encryption algorithm with the private key of the sender.
The output of this function will be both the encrypted hash value of the message using RSA, and the plaintext (original content of message.txt file).
Note here that in this scenario only message integrity is required, and no confidentiality will be provided. This is why the message is sent in plain.
 

2) Create a receiver function the does the following:

The input is the public key of the sender, the encrypted hash value, and the plaintext.
Decrypt the received hash value using the public key of the sender.
Calculate the hash value of the received plaintext.
Compare the hash value of the received plaintext with the hash value of the decrypted received hash.
 

3) perform a test that include two cases:

in first case, the message in not altered, which means that the receiver after checking will accept the received message.
In second case, the message will be modified and then fed to the receiver function. In this case, the modification will be spotted and the received message will be rejected by the receiver.


Some of the functions that were coded in the practice lecture and might be useful for this assignment.

In [55]:
from sympy import randprime
from math import gcd

def egcd(a,b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        return (g, y - (b // a) * x, x)

def modinv(b, m):
    g, x, _ = egcd(b, m)
    if g == 1:
        return x % m

In [56]:
def generatePrimes(key_size):
    prime_1 = 0
    prime_2 = 0
    while prime_1 == prime_2 or(prime_1 *prime_2) > 2**key_size:
        prime_1 = randprime(3, 2**key_size/2)
        prime_2 = randprime(3, 2**key_size/2)
    return prime_1, prime_2

In [57]:
def generate_public_Exponents(totient):
    public_exponent = 0
    for e in range(3,totient-1):
        if gcd(e,totient) == 1:
            public_exponent = e
            break
    return public_exponent

In [58]:
def generateTotient(prime_1, prime_2):
    return (prime_1 - 1) * (prime_2 - 1)

In [59]:
def rsa_encrypt(plain_text,rsa_modulus, public_exponent):
    cipher = ''.join(chr((ord(ch)**public_exponent)%rsa_modulus) for ch in plain_text)
    return cipher.encode().hex()

In [60]:
def rsa_decrypt(cipher_text, rsa_modulus, private_exponent):
    return ''.join(chr((ord(ch)**private_exponent)%rsa_modulus)
                   for ch in bytearray.fromhex(cipher_text).decode())

In [61]:
from hashlib import sha1

def hashFileContentWithSha1(fileContent):
    return sha1(fileContent.encode()).hexdigest()

In [62]:
def readFileContent(fileName):
    with open(fileName, 'r') as file:
        return file.read()

Part 1 - Sender. Sender takes fileContent hashes it, encrypts it with private key and returns encrypted hash and fileContent.

In [63]:
def senderRsa(fileName, privateKey):
    (rsa_modulus, private_exponent) = privateKey
    fileContent = readFileContent(fileName)
    hashFileContent = hashFileContentWithSha1(fileContent)
    encryptedHash = rsa_encrypt(hashFileContent, rsa_modulus, private_exponent)
    return (fileContent, encryptedHash)

Part 2 - Receiver. Receiver takes public key, encrypted hash and fileContent. It decrypts the encrypted hash with public key, hashes the fileContent and compares the two hashes. If they are equal, it returns prints "Hashes are equal", otherwise "Hashes are not equal".

In [64]:
def receiverRsa(fileContent, encryptedHash, publicKey):
    (rsa_modulus, public_exponent) = publicKey
    hashedFileContent = hashFileContentWithSha1(fileContent)
    decryptedHash = rsa_decrypt(encryptedHash, rsa_modulus, public_exponent)
    if hashedFileContent != decryptedHash:
        print("Hashes are not equal")
    else:
        print("Hashes are equal")

 Public and private keys generation for a given key size.

In [65]:
def generateKeys(key_size):
    prime_1,prime_2 = generatePrimes(key_size)
    totient = generateTotient(prime_1, prime_2)
    public_exponent = generate_public_Exponents(totient)
    private_exponent = modinv(public_exponent, totient)
    rsa_modulus = prime_1 * prime_2 # integer n from the slides
    public_key = (rsa_modulus, public_exponent)
    private_key = (rsa_modulus, private_exponent)
    return (public_key, private_key)

key_size = 10
public_key, private_key = generateKeys(key_size)

In [66]:
# part 1
(fileContent, encrypterHash) = senderRsa('message.txt', private_key)
print("File content: ", fileContent)
print("Encrypted hash: ", encrypterHash)

File content:  hello world from Budapest

Encrypted hash:  c290c3b2c3a3c3becab8cab8cf9ac3beca90ca90cf9ac295cbbdc2955ec3b2cf9ac3b25ecb80cf9ac290cb80c4a2cbbdce8ec290c290ca90cbbd5ec290ce8ecdbfc3b2cbbdcb80cdbfcb805e


In [67]:
# part 2
receiverRsa(fileContent, encrypterHash, public_key)

Hashes are equal


In the part 3 of the assignment, you have to alter the message and check if the receiver will see that hashes changed or not. 

In [68]:
# part 3
# not altered message
receiverRsa(fileContent, encrypterHash, public_key)

# altered message
fileContent = fileContent.replace('B', 'b')
print("Altered file content: ", fileContent)
receiverRsa(fileContent, encrypterHash, public_key)

Hashes are equal
Altered file content:  hello world from budapest

Hashes are not equal


In [76]:
## diffie hellman
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

parameters = dh.generate_parameters(generator=2, key_size=512)

alice_private_key = parameters.generate_private_key()
bob_private_key = parameters.generate_private_key()
bob_public_key = bob_private_key.public_key()
alice_public_key = alice_private_key.public_key()

alice_shared_key = alice_private_key.exchange(bob_public_key)

alice_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(alice_shared_key)

print (alice_derived_key)

## bob
bob_shared_key = bob_private_key.exchange(alice_public_key)

bob_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(bob_shared_key)

print (bob_derived_key)

if alice_derived_key == bob_derived_key:
    print ("derived keys are equal")
else:
    print ("derived keys are not equal")

b'RT@O\x83\x82Zr\x91\x9f\x13*h\x1f\x00\x9fO\x86\xcb\xa6\xa1\x1f\xecu\x05m\xb5\xe2\x80\x84`\xf6'
b'RT@O\x83\x82Zr\x91\x9f\x13*h\x1f\x00\x9fO\x86\xcb\xa6\xa1\x1f\xecu\x05m\xb5\xe2\x80\x84`\xf6'
derived keys are equal


For AES, the legal key sizes are 128, 192, and 256 bits.
for des it was 64 - but 60 bits are used for the key and 4 bits are used for parity checking

In [83]:
parameters = dh.generate_parameters(generator=2, key_size=512)
p = parameters.parameter_numbers().p
g = parameters.parameter_numbers().g
client_private_key = parameters.generate_private_key()
client_y = client_private_key.public_key().public_numbers().y
client_x = client_private_key.private_numbers().x
print (p)
print (g)
print (client_y)
print (client_x)
# it stores p,g,client_y,client_x as the main parameters for later use
# then it sends p,g,client_y parameters to the server



13195430907065982499259062465410625803646766784897984013508805312425762358788088613294355083082803471651919549194854967705644355664737625734230481600148447
2
2090203683818023564087866247877183478168688996525202429929605747435682669462723352688462415360381145802883875119734170534491304983628973428590607460264898
6356093229651920131717983900732712866487482268325586104467852338167596444915807582867946366631377096138824427303071253420571294934634360181659938083616359


In [84]:
dominykas_key = 4406081192861135331958262273451869593683017678524489026964821445397841449792156988203193189716958862372703950153564671359780188501688643780552798404389598

In [85]:
pn = dh.DHParameterNumbers(p, g)
client_public_number = dh.DHPublicNumbers(client_y, pn)
client_public_number = dh.DHPublicNumbers(dominykas_key, pn)
client_public_key = client_public_number.public_key()

parameters = pn.parameters()
server_private_key = parameters.generate_private_key()
server_y = server_private_key.public_key().public_numbers().y
server_x = server_private_key.private_numbers().x
print (server_y)
print (server_x)

7536933467176977867876905576594897307915661342613611357121492835155498369306473596391913103632464543902546600847724424435622710878828202978567924386020210
4361460398999010021971883749517844170616993542548293292334868452466970905747308270850229603648619925087920801878581986273258800062747298257582663387768460


In [86]:
pn = dh.DHParameterNumbers(p, g)
client_public_number = dh.DHPublicNumbers(client_y, pn)
client_private_number = dh.DHPrivateNumbers(client_x, client_public_number)
client_private_key = client_private_number.private_key()



server_public_number = dh.DHPublicNumbers(server_y, pn)
server_public_key = server_public_number.public_key()

In [87]:
client_shared_key = client_private_key.exchange(server_public_key)

client_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(client_shared_key)

print (client_derived_key)

## bob
server_shared_key = server_private_key.exchange(client_public_key)

server_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(server_shared_key)

print (server_derived_key)

if client_derived_key == server_derived_key:# check this for the next time
    print ("derived keys are equal")

b'*-e\r\x9b\xe8\x15g\x07\xbae\xfb\x8e>\xc1a\x1d5\x88\xdf\xf7\xab\xc5\xb8\xc1\x0ea\xe3t\x1b\xad\xc5'
b"\xfa8\x82\x98\xde\x18N7x'\x97+)\xd6\x0cl\x90\xb5\x1b*\x9dX\x13\x85\x9c\x18JF\xba\xc1\x80\x89"


In [88]:
# can use our onw code and presentations
pn = dh.DHParameterNumbers(p, g)

client_public_number = dh.DHPublicNumbers(client_y, pn)
client_private_number = dh.DHPublicNumbers(client_x, client_public_number)
client_private_key = client_private_number.private_key()

server_public_number = dh.DHPublicNumbers(server_y, pn)
server_public_key = server_public_number.public_key()

client_shared_key = client_private_key.exchange(server_public_key)

derived_key_client = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(client_shared_key)
print(derived_key_client)

TypeError: parameters must be an instance of DHParameterNumbers.